In [6]:
import subprocess
import yaml

import sys
import os
import re
import numpy as np
import scipy.io as sio
import imageio
import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter


# If your script is in a subfolder, navigate one directory up
parent_dir = os.path.dirname(os.getcwd())

# Add the parent directory to sys.path
sys.path.append(parent_dir)

# from utlis.projection import *                                 
from utlis.com_traga_utlis import load_data, plot_3d_trajectory, detect_jumps, generate_jump_video#*
# from utlis.com_traga_utlis import plot_com_all, process_folders, temp_change_calib_pos_to_0, generate_dannce_vid_seq# *

In [ ]:
#below are just final steps of optimization.... usually in the beginning i will just predict many 100 using the integrated debugging function in yaml file

def com_single_vis(base_folder, com_foler):
    # weired_folders = [
    # # "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_07_03/1691486_left_right_caffeine_1448",
    # # "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_07_03/1691486_left_caffeine_1051",
    # # "/home/lq53/mir_data/24summ/2024_07_10/1691485_no_hole_habituation_13_59"
    # "/home/lq53/mir_data/24summ/2024_06_26/1686940_left"
    # ]
    # for wie in weired_folders:
        # /home/lq53/mir_data/24summ/2024_06_26/1686940_left/COM_df/predict_results//vis/vis_combined_Camera2_1000_from_0.mp4
    # com_foler = os.path.join(wie, 'COM_df/predict_results/df_labels_240816_6comsets_continued')
    com_path = os.path.join(com_foler, 'com3d.mat')   
    com_folder_save = os.path.join(com_foler, 'vis')
    if not os.path.exists(com_folder_save):
        os.makedirs(com_folder_save)
    graph_title = "z_com3d_vis_"
    
    com_data = load_data(com_path)

    plot_3d_trajectory(com_data, graph_title, com_folder_save)
    jump_indices = detect_jumps(com_data, com_folder_save)

    # # # produce video, which is not necessary if not labeling more com to detect what's wrong


    save_path = com_folder_save # os.path.join(com_foler, 'vis') #os.path.join(pred_path, 'vis')
    # if not os.path.exists(save_path):
    #     os.makedirs(save_path)
    # base_folder = wie
    base_base_folder = os.path.dirname(base_folder)
    generate_jump_video(com_data, base_folder, jump_indices, graph_title, save_path, cam='Camera1')


def update_yaml(path, param_changes):
    with open(path, 'r') as file:
        config = yaml.safe_load(file)
    
    # Apply the parameter changes
    config.update(param_changes)
    
    # Save the updated configuration back to the same file
    with open(path, 'w') as file:
        yaml.safe_dump(config, file)

def COM_train_pred_vis(base_fold, io_param_changes, param_changes=None):
    com = 'com'
    # dannce = 'dannce' #actually dannce probably will be different so lets do this later

    io_name = 'mir_io.yaml'
    io_path = os.path.join(base_fold, io_name)

    com_config_name = f'mir_{com}_mouse_config.yaml'
    com_config_path = os.path.join(base_fold, com_config_name)

    update_yaml(io_path, io_param_changes)
    print('updated io file')

    if param_changes is not {}:
        update_yaml(com_config_path, param_changes)
        print('updated com_config file')

    print("Starting training...")
    subprocess.run(f"com-train {com_config_path}", shell=True, check=True, text=True)
    print("Training completed.")

    print("Starting prediction...")
    subprocess.run(f"com-predict {com_config_path}", shell=True, check=True, text=True)
    print("Prediction completed.")

    print('start visulizing')
    com_single_vis(base_fold, io_param_changes['com_predict_dir'])
    print(f'finished, check results in{}')



    


In [ ]:
base_fold = "/home/lq53/mir_data/24summ/2024_06_26/1686940_left"

# new_train_dir_name = './COM_df_z/train_results/df_z_240816_6comsets_fromdemo/'
param_changes = {
    # 'max_num_samples': 1000 # 'max' 
}

io_param_changes = {
    'com_train_dir': './COM_df_z/train_results/df_z_240816_6comsets_fromdemo/', # new_train_dir_name,  # Example: change the learning rate
    'com_predict_dir': './COM_df_z/predict_results/df_z_240816_6comsets_fromdemo/',        # Example: change the batch size
    # 'com_predict_weights': 
    # 'com_file': 
    # dannce_predict_model
    # dannce_train_dir: ./DANNCE/train_results/df_240809_mirlabelall_2nd100epo_opti/
# dannce_predict_dir: ./DANNCE/predict_results/240812_ssh_2bestwalk/
}




COM_train_pred_vis(base_fold)